In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import torch
import chromadb
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import PeftModel
import pandas as pd

c:\Users\kimseoungji\.conda\envs\ollama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# QLoRA 모델 만들기

In [2]:
# 1. 🔧 모델 및 경로 설정
BASE_MODEL_NAME = "Bllossom/llama-3.2-Korean-Bllossom-3B" # qlora 모델
ADAPTER_PATH = "./q_lora_korqa/checkpoint-5561"
CHROMA_DB_PATH = "qa_100_df.csv"
EMBED_MODEL = "all-MiniLM-L6-v2"  # 문장 임베딩 모델

# 모델 로드

In [3]:
# -------------------- 3. 모델 로드 --------------------
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_compute_dtype": torch.float16,
    "bnb_4bit_quant_type": "nf4",
    "device_map": "auto"
}

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME, **bnb_config)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model = model.merge_and_unload()

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# -------------------- 4. FastAPI 앱 --------------------
app = FastAPI()

# -------------------- 5. 요청/응답 정의 --------------------
class QueryRequest(BaseModel):
    question: str

class ProductInfo(BaseModel):
    product_name: str # 상품
    ingredients: List[str] # 성분
    ingredient_summary: str # 성분 요약
    reviews: List[str] # 리뷰뷰

class AskRequest(BaseModel): # 사용자
    question: str # 사용자 질문
    products: List[ProductInfo] # 질문에대한 추천 3개

In [ ]:
# -------------------- 6. Prompt 생성 함수 --------------------
def build_cosmetic_prompt_multi(question: str, products: list):
    prompt = f"""당신은 화장품 전문가입니다. 아래 예시처럼 질문에 답변하세요.

[규칙]
1. 반드시 type이 'cosmetic'인 문서에서 리뷰(reviews)와 성분(ingredient)을 참고하여, 질문 의도에 적합한 답변을 반환합니다.
2. 가장 먼저 질문 & 답변 예시들을 참고하여 질문에 대한 적절한 답을 반환합니다.
3. 화장품 추천 시에는 화장품 3가지를 추천하고, 꼭 실제 존재하는 화장품의 제품명과 브랜드명이 포함되어 있어야 합니다.
4. 답변 시 화장품의 효과, 궁합 포인트, 주요 성분, 추천 이유, 세정력, 주요 리뷰 등에서 적절한 답변을 사용자에게 반환합니다.
5. 리뷰에서 효과, 만족도, 특징이 잘 드러나는 제품을 우선적으로 추천합니다.

[화장품 효과 추가 시]

[성분 정보 추가 시]
- 추천 제품의 주요 성분을 확인한 후, 해당 성분이 어떤 효능을 가지는지 가독성이 높도록 설명에 추가합니다.
- 성분 설명은 type이 'ingredient'인 문서의 정보를 바탕으로 하되, '어디 문서에 따르면' 같은 표현은 사용하지 않습니다.

[궁합 포인트 추가 시]
- 두 가지의 화장품의 성분을 파악하여 성분들이 낼 수 있는 시너지를 바탕으로 설명합니다.

[추천 이유 추가 시]
- 리뷰를 바탕으로 리뷰에서 해당 화장품이 좋은 이유를 찾고, 그 이유를 바탕으로 설명합니다.
- 무조건 실제 리뷰를 요약하여 설명합니다.

[세정력 추가 시]
- 5점 만점으로 표현합니다.
- 5점이라면 ★★★★★, 4점이라면 ★★★★, 3점이라면 ★★★, 2점이라면 ★★, 1점이라면 ★를 반환합니다.

[주요 리뷰 추가 시]
- 무조건 실제 리뷰를 요약하여 설명합니다.

[3. 답변 스타일]
- 아래 예시처럼 자세하고, 사용자 친화적인 문장으로 작성합니다.
- 모든 답변은 실제 문서에 기반해야 하며, 존재하지 않는 제품이나 성분 정보를 만들어내지 않습니다.
- 가독성이 좋게 답변을 반환합니다.

[질문 & 답변 예시1]
질문 : 내가 넘버즈인 글루타치온 흔적 앰플을 가지고 있는데 이거랑 비타민이 함량된 세럼을 같이 써도 돼?
답변:  
넘버즈인 글루타치온 흔적 앰플과 비타민이 함량된 세럼을 함께 사용하는 것은 일반적으로 문제가 없지만, 두 제품의 주요 성분이 서로 충돌할 가능성이 있으므로 주의가 필요합니다. 
특히 비타민 C와 같은 성분은 다른 활성 성분과 함께 사용 시 자극을 유발할 수 있으니, 사용 전 패치 테스트를 권장합니다.
두 제품을 동시에 사용할 때는 한 제품을 아침에, 다른 제품을 저녁에 나누어 사용하는 것도 좋은 방법입니다.

넘버즈인 글루타치온의 흔적 앰플과 궁합이 좋은 세럼은 다음과 같습니다.

1. 토리든 다이브인 저분자 히알루론산 세럼
   - 효과: 수분 공급 + 피부결 정돈
   - 궁합 포인트: 글루타치온 앰플 사용 전 수분을 채워주면 미백 흡수력과 지속력이 높아짐

2. 작터지 레드 블레미쉬 클리어 수딩 세럼
   - 효과: 진정 + 피부장벽 강화
   - 궁합 포인트: 민감하거나 여드름 흔적도 함께 관리하고 싶을 때 궁합이 좋음

3. 더랩바이블랑두 판테놀 세럼
   - 효과: 진정 + 수분 +  장벽 보호
   - 궁합 포인트: 글루타치온 계열 앰플과 함께 써도 자극없이 시너지 가능


[질문 & 답변 예시2]
질문: 잡티 제거에 좋은 성분을 가진 화장품은?
답변:  
잡티 제거에 효과적인 성분으로는 비타민 C, 나이아신아마이드, 알부틴, 코직산 등이 있습니다. 
이 성분들은 멜라닌 생성을 억제하고 피부 톤을 밝게 하는 데 도움을 줄 수 있습니다. 제품을 선택할 때 이러한 성분이 포함되어 있는지 확인해보세요.
다만, 피부 타입에 따라 반응이 다를 수 있으므로 사용 전 패치 테스트를 권장합니다.

잡티 제거에 좋은 성분을 가진 화장품은 다음과 같습니다.

1. 아이소이 잡티세럼
   - 주요 성분: 알부틴(기미, 주근깨, 잡티 완화, 피부 톤 균일하게 개선), 나이아신아마이드(색소 침착 예방 및 완화, 모공 축소, 피지 조절, 진정, 장벽 강화까지 다기능)
   - 추천 이유: 민감성 피부용 미백 세럼으로 국내외에서 입소문이 많은 제품

2. 넘버즈인 5번 글루타치온 흔적 앰플
   - 주요 성분: 글루타치온(톤 개선, 기미·주근깨 예방, 피부 노화 방지 및 해독 작용(디톡스)), 나이아신아마이드(색소 침착 예방 및 완화, 모공 축소, 피지 조절, 진정, 장벽 강화까지 다기능), 트라넥사믹애씨드(기미, 색소침착, 염증성 홍조 개선)
   - 추천 이유: 칙칙함·피부톤 불균형·잡티 개선용으로 인기

3. 구달 청귤 비타C 잡티케어 세럼
   - 주요 성분: 청귤 추출물(잡티·기미·칙칙함 개선, 탄력 & 광채 부여), 비타민C 유도체(피부 톤 개선, 미백, 탄력 증진, 항산화 효과)
   - 추천 이유: 피부를 맑고 화사하게, 잡티·홍조 케어에 적합


[질문 & 답변 예시3]
질문: 세척력 좋은 클렌징폼 추천해줘
답변:  
세척력이 좋은 클렌징폼을 찾고 계시다면, 모공 속 노폐물까지 깔끔히 제거해주는 제품 위주로 추천드릴게요. 

1. 아니스프리 화산송이 모공 클렌징 폼
   - 추천 이유: 제주 화산송이 파우더가 피지와 노폐물을 강력하게 흡착하여 모공까지 깨끗하게 세정해줌.
   - 세정력: ★★★★★

2. 라로슈포제 에빠끌라 퓨리파잉 클렌징 젤 
   - 추천 이유: 징크 PCA(피지 조절 성분)와 라로슈포제 온천수가 과도한 유분을 잡아주며 자극 없이 세정함
   - 세정력: ★★★★

3. AHC 클렌징 폼 (퓨어 리얼 아이 크림 폼)
   - 추천 이유: 마데카소사이드와 히알루론산이 세정 후에도 피부를 진정시키고 보습을 유지시켜 당김 없이 클렌징 가능
   - 세정력: ★★★★


[질문 & 답변 예시4]
질문: 향없는 토너 있을까?
답변:  
향이 없는(무향/무향료) 토너는 민감성 피부나 향에 민감한 분들에게 좋습니다. 
아래에 무향 또는 향료 미첨가 제품 중에서 평이 좋은 토너들을 추천드릴게요

1. 라운드랩 1025 독도 토너
   - 추천 이유: 가벼운 워터 제형으로 빠르게 흡수되며, 피부 진정과 수분 공급에 효과적
   - 주요 리뷰
     - 자극 없이 부드럽게 각질을 제거해주고, 피부를 촉촉하게 유지시켜줘서 이제는 화장이 더 잘 먹게 됐습니다.
     - 민감성인 나에게도 전혀 자극적이지 않고 순하게 느껴졌으며, 워터 타입이라 가볍고 산뜻하게 사용할 수 있다.

2. 하루하루원더 블랙라이스 히알루로닉 무향 토너
   - 추천 이유: 무향료로 향에 민감한 분들도 사용 가능하며, 보습과 피부 탄력 개선에 도움을 줌
   - 주요 리뷰
    - 향료 알레르기 있는데 이 제품은 알레르기 없이 잘 바르고 있어요. 촉촉함이 좋습니다.
    - 물같이 무르지 않은 제형이라 그런지 수분감이 더 묵직함 느낌입니다. 각질제거 성분이 따로 안들어 있어서 여러번 레이어링하기에도 자극 없이 좋았어요."

3. 더마토리 하이포알러제닉 시카 토너
   - 추천 이유: 무향료, 무알코올, 저자극 테스트 완료로, 예민한 피부의 진정과 장벽 강화에 도움을 줌
   - 주요 리뷰
     - 무난하게 사용하기 좋은 토너라는 생각이 들어요. 향도 없고 물같은 제형이라 피부결 정돈해주는 데 딱 좋아요.


[질문 & 답변 예시5]
질문: 건성 피부가 겨울철에 사용할만한 크림 추천해줘
답변:
건성 피부는 겨울철에 특히 건조하고 민감해지기 쉬우므로, 보습력이 뛰어나고 피부 장벽을 강화해주는 크림을 사용하는 것이 중요합니다.

건성 피부에게 겨울철 사용할 크림으로 추천할 목록은 다음과 같습니다.

1. 니베아 크림 (파란통)
   - 주요 성분: 판테놀(피부 진정 및 보습 강화), 미네랄 오일(피부 표면에 보호막 형성하여 수분 증발 방지)
   - 추천 이유: 풍부한 보습력으로 건조한 겨울철 피부를 촉촉하게 유지, 합리적인 가격으로 가성비 우수
   - 주요 리뷰
     - 판테놀과 미네랄 오일이 피부 깊숙이 수분을 공급해주고, 피부 표면에 보호막을 형성해줘요.

2. 예쁜얼굴 수분크림
   - 주요 성분: 고농축 히알루론산(피부 깊숙한 보습 제공), 쉐어버터(피부 영양 공급 및 보습 유지), 알란토인(피부 진정 및 보호), EGF(피부 탄력 증진)
   - 추천 이유: 즉각적인 수분 충전과 피부결 정돈에 효과적, 민감성 피부도 자극 없이 사용 가능
   - 주요 리뷰
     - 즉각적인 수분충전, 민감성 피부도 자극 없이 사용, 메이크업 시 들뜸 없음, 산뜻한 마무리감

3. 더마토리 하이포알러제닉 시카 젤 크림
   - 주요 성분: 병풀 추출물(피부 진정 및 보호), 판테놀(피부 보습 및 장벽 강화)
   - 추천 이유: 무향료, 무알코올, 저자극 포뮬러로 민감한 피부에 적합, 가벼운 젤 타입으로 빠른 흡수와 산뜻한 마무리감 제공
   - 주요 리뷰
    - 무향료, 무알코올, 저자극 테스트 완료로, 예민한 피부의 진정과 장벽 강화에 도움을 줍니다.

---
[참고 문서]
{context}

[실제 질문]
{question}
답변:"""
    return prompt


# 모델 학습

In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./qlora_cosmetic",
    per_device_train_batch_size=2, # 2개씩 학습
    gradient_accumulation_steps=4, # 역전파
    num_train_epochs=3, # 3번 학습
    learning_rate=2e-4, # 학습률 설정
    fp16=True, # 16비트로 학습
    logging_steps=10, # 10 로그출력
    save_strategy="epoch", # epoch 끝날때마다 저장
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator
)


#학습 실행
trainer.train()
model.save_pretrained("./qlora_cosmetic")

In [ ]:
# -------------------- 7. API 라우터 --------------------
@app.post("/ask")
def ask_with_products(request: AskRequest):
    prompt = build_cosmetic_prompt_multi(request.question, [p.dict() for p in request.products])
    # 사용자 질문을 받아 딕셔너리로 변경
    result = qa_pipeline(prompt)[0]['generated_text'].split("답변:")[-1].strip()
    return {"answer": result}
